In [ ]:
import numpy as np
import pandas as pd
     

#!pip install simpletransformers
from simpletransformers.classification import ClassificationModel

In [ ]:
dt= pd.read_csv("/content/teknofest_train_final.csv",sep="|")
dt.head()
     

,id,text,is_offensive,target
0,81c11060-a240-4d54-841b-9e2916039e85,çürük dişli,1,INSULT
1,be80ebbf-b322-4c3b-afa1-94932ea80731,Bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST
2,f99e2513-83ed-4076-ac72-b9e2cff3f049,erkekler zora gelmez,1,SEXIST
3,83ed2b2e-b815-4f36-9fc4-80a9050cf2d0,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER


loodos/bert-base-turkish-uncased

In [ ]:
dt["target"].unique()

array(['INSULT', 'RACIST', 'SEXIST', 'PROFANITY', 'OTHER'], dtype=object)

In [ ]:
dt["labels"] = pd.factorize(dt.target)[0]

In [ ]:
dt

,id,text,is_offensive,target,labels
0,81c11060-a240-4d54-841b-9e2916039e85,çürük dişli,1,INSULT,0
1,be80ebbf-b322-4c3b-afa1-94932ea80731,Bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST,1
2,f99e2513-83ed-4076-ac72-b9e2cff3f049,erkekler zora gelmez,1,SEXIST,2
3,83ed2b2e-b815-4f36-9fc4-80a9050cf2d0,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY,3
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER,4
...,...,...,...,...,...
12612,71eedfa1-8fa6-425c-b982-258c3b29c003,uyuma taklidi yapan tehlikeli bir hayvanın göz...,0,OTHER,4
12613,b38eed16-6501-4563-8b33-ff2e634bb8e5,yolda at kavga eden üç oğlan çocuğu görür,0,OTHER,4
12614,c8a051a8-94ef-4b64-a48e-54d0fa4f8323,sizin köpeklerinizin burnu bile daha iyi koku ...,0,OTHER,4
12615,513a7e6d-4207-4a16-9b47-972f26e23cfe,hayalleri gerçek etmek için birisinin delilik ...,0,OTHER,4


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dt, test_size=0.2, random_state=42)
     

train=train[["text","labels"]]
test=test[["text","labels"]]
     

#for bert text = string      label = int
train["text"]=train["text"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

--------------------------------------------------------------------------------

In [ ]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-128k-cased', num_labels=5, use_cuda=False,args={'reprocess_input_data': True, "output_dir": "bert_model"})
model.train_model(train)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

  0%|          | 0/10093 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1262 [00:00<?, ?it/s]

(1262, 0.4373161095662578)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

predictions = model_outputs.argmax(axis=1)
     
actuals = test.labels.values

  0%|          | 0/2524 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/316 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.9342313787638669

In [ ]:
test

,text,labels
10137,esasen bunun böyle olacağı da bize sekreterya ...,4
5099,örümcek kafalı bu başını kapatanlar saçlarını ...,1
10848,bu gerçekten mümkün değil,4
6310,gavur eti yenmez,1
2439,Ne kadar az siklersen o kadar çok mutlu olursun,3
...,...,...
1183,Benim aklıma gelip başkasına gidişini sikeyim,3
5665,orul& orul orospu< çocukları-,3
7729,her şeye sıfırdan başladık,4
7391,Dana= gibi. olmuşsun,0


In [ ]:
dene = test.iloc[752]['text']
print(dene)
     
tahmin=model_from_pickle.predict([dene])


kıl herif gibi davranma


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tahmin

(array([0]),
 array([[ 4.82061625, -2.2739048 ,  0.06289925, -0.11990602, -2.72607923]]))

In [ ]:
dt[(dt["text"]=="kıl herif gibi davranma")==True]

,id,text,is_offensive,target,labels
10695,7048769a-8518-47ec-8978-8496c6dd9556,kıl herif gibi davranma,1,INSULT,0
